In [ ]:
from threading import Thread,Event
import time
def countdown(n,started_evt):
    print('countdown starting')
    started_evt.set()
    while n > 0:
        print('T-minus',n)
        n -= 1
        time.sleep(5)
started_evt = Event()
print('Launching countdown')
t = Thread(target=countdown,args=(10,started_evt))
t.start()
started_evt.wait()
print('countdown is running')

In [ ]:
import threading
import time
class PeriodicTime:
    def __init__(self,interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()
    def start(self):
        t = threading.Thread(target=self.run,daemon=True)
        t.start()
    def run(self):
        while True:
            time.sleep(self._interval)
            with self._cv:
                self._flag ^= 1
                self._cv.notify_all()
    def wait_for_tick(self):
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()
ptimer = PeriodicTime(5)
ptimer.start()
def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minux',nticks)
        nticks -= 1
def countup(last):
    n = 0
    while n <last:
        ptimer.wait_for_tick()
        print('counting',n)
        n += 1
threading.Thread(target=countdown,args=(10,)).start()
threading.Thread(target=countup,args=(5,)).start()

In [ ]:
from queue import Queue
from threading import Thread,Event

In [ ]:
class ActorExit(Exception):
    pass
class Actor:
    def __init__(self):
        self._mailbox = Queue()
    def send(self,msg):
        self._mailbox.put(msg)
    def recv(self):
        msg = self._mailbox.get()
        if msg is ActorExit:
            raise ActorExit()
        return msg
    def close(self):
        self.send(ActorExit)
    def start(self):
        self._terminated = Event()
        t = Thread(target=self._bootstrap)
        t.daemon = True
        t.start()
    def _bootstrap(self):
        try:
            self.run()
        except ActorExit:
            pass
        finally:
            self._terminated.set()
    def join(self):
        self._terminated.wait()
    def run(self):
        while True:
            msg = self.recv()
class PrintActor(Actor):
    def run(self):
        while True:
            msg = self.recv()
            print('Got:',msg)

In [ ]:
p = PrintActor()

In [ ]:
p.start()

In [ ]:
p.send('hello')

In [ ]:
p.send('world')

In [ ]:
p.close()

In [ ]:
p.join()

In [24]:
import requests
url = 'http://113.107.161.47:8081/api/counters'
header = {'Content-Type':'application/x-www-form-urlencoded'}
parms = {
    'limit':50000,
    "endpoints":'["113.107.167.139"]'
}
resp = requests.post(url,data=parms,headers=header)
ret = resp.text

In [25]:
dic = json.loads(ret)

In [26]:
len(dic['data'])

589

In [28]:
from urllib import request,parse
url = 'http://httpbin.org/post'
parms = {
    'name1':'value1',
    'name2':'value2'
}
header = {
    'User-agent': 'none/ofyourbusiness',
    'Content-Type':'application/x-www-form-urlencoded',
    'Spam':'Eggs'
}
querystring = parse.urlencode(parms)
req = request.Request(url,querystring.encode('ascii'),headers=header)
u = request.urlopen(req)
u.read()

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Connect-Time": "0", \n    "Connection": "close", \n    "Content-Length": "25", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "Spam": "Eggs", \n    "Total-Route-Time": "0", \n    "User-Agent": "none/ofyourbusiness", \n    "Via": "1.1 vegur", \n    "X-Request-Id": "a38d66c1-2edb-4ef5-9c71-9903b8c40657"\n  }, \n  "json": null, \n  "origin": "39.109.124.40", \n  "url": "http://httpbin.org/post"\n}\n'